<a href="https://colab.research.google.com/github/AlbertoVentura8/LimpiezaDatos/blob/main/MarchMad_Limpieza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**En este apartado, me voy a centrar en la limpieza del dataset.**


1.   Voy a dar acceso a un drive donde está almacenado y donde almacenaremos el
resultado del dataset.
2.   Eliminaremos una serie de columnas que hacen que el conjunto de datos tenga una dimensión poco manejable, y lo vamos a reducir para que sea tratable y poder sacar algunas conclusiones.

In [1]:
# Vamos a montar el drive, lo primero es
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [2]:
#Hacemos la carga del dataset. Dataset previamente subido a GoogleColab.
import pandas as pd
NCAA_df = pd.read_csv("https://drive.google.com/uc?export=download&id=1pkvH-wKa3nLWcquQNNv4GxcDfMc3NOwo")
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7950, 144)


,Season,Team Name,Conference,Historical Conference,Adjusted Temo,Adjusted Tempo Rank,Raw Tempo,Raw Tempo Rank,Adjusted Offensive Efficiency,Adjusted Offensive Efficiency Rank,...,RankPGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating,Net Rating Rank,Seed,Region,Post-Season Tournament,Post-Season Tournament Sorting Index
0,2015,Kentucky,Southeastern Conference (SEC),Southeastern Conference (SEC),62.4,274,63.8,242,121.3,6,...,311.0,John Calipari,14 years,15.0,36.9,1,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
1,2021,Gonzaga,West Coast Conference (WCC),West Coast Conference (WCC),73.8,7,74.3,14,126.4,1,...,20.0,Mark Few,24 years,25.0,36.5,2,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
2,2024,UConn,Big East Conference,Big East Conference,64.6,330,66.0,305,127.5,1,...,126.0,Dan Hurley,5 years,6.0,36.4,3,1,East,March Madness,1
3,2008,Kansas,Big 12 Conference,Big 12 Conference,66.8,109,68.5,120,120.9,2,...,209.0,Bill Self,20 years,21.0,35.2,4,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
4,2019,Virginia,Atlantic Coast Conference (ACC),Atlantic Coast Conference (ACC),59.4,353,60.6,353,123.4,2,...,196.0,Tony Bennett,14 years,15.0,34.2,5,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5


In [3]:
#En esta línea eliminamos todas la columnas que tengan la palabra Raw, para reducir el dataset inicial.
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Raw')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()


Tamaño del dataset: (7950, 138)


,Season,Team Name,Conference,Historical Conference,Adjusted Temo,Adjusted Tempo Rank,Adjusted Offensive Efficiency,Adjusted Offensive Efficiency Rank,Adjusted Defensive Efficiency,Adjusted Defensive Efficiency Rank,...,RankPGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating,Net Rating Rank,Seed,Region,Post-Season Tournament,Post-Season Tournament Sorting Index
0,2015,Kentucky,Southeastern Conference (SEC),Southeastern Conference (SEC),62.4,274,121.3,6,84.4,1,...,311.0,John Calipari,14 years,15.0,36.9,1,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
1,2021,Gonzaga,West Coast Conference (WCC),West Coast Conference (WCC),73.8,7,126.4,1,89.9,11,...,20.0,Mark Few,24 years,25.0,36.5,2,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
2,2024,UConn,Big East Conference,Big East Conference,64.6,330,127.5,1,91.1,4,...,126.0,Dan Hurley,5 years,6.0,36.4,3,1,East,March Madness,1
3,2008,Kansas,Big 12 Conference,Big 12 Conference,66.8,109,120.9,2,85.7,1,...,209.0,Bill Self,20 years,21.0,35.2,4,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
4,2019,Virginia,Atlantic Coast Conference (ACC),Atlantic Coast Conference (ACC),59.4,353,123.4,2,89.2,5,...,196.0,Tony Bennett,14 years,15.0,34.2,5,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5


In [4]:
#Hacemos lo mismo que en la línea anterior para eliminar los rankings.
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Rank')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7950, 75)


,Season,Team Name,Conference,Historical Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,...,SGDR,PGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating,Seed,Region,Post-Season Tournament,Post-Season Tournament Sorting Index
0,2015,Kentucky,Southeastern Conference (SEC),Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,...,13.00,10.99,John Calipari,14 years,15.0,36.9,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
1,2021,Gonzaga,West Coast Conference (WCC),West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,...,24.63,19.49,Mark Few,24 years,25.0,36.5,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
2,2024,UConn,Big East Conference,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,...,20.55,15.33,Dan Hurley,5 years,6.0,36.4,1,East,March Madness,1
3,2008,Kansas,Big 12 Conference,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,...,13.90,12.87,Bill Self,20 years,21.0,35.2,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5
4,2019,Virginia,Atlantic Coast Conference (ACC),Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,...,17.84,13.59,Tony Bennett,14 years,15.0,34.2,Not In a Post-Season Tournament,Not In a Post-Season Tournament,Not In a Post-Season Tournament,5


In [5]:
# Con estas dos limpiezas, ya hemos pasado de un dataset de 144 columnas, a uno de 75
# Vamos a ver los tipos de nuestras columnas
print(NCAA_df.dtypes)
tipo_resumen = NCAA_df.dtypes.value_counts()
print("Tipos de Datos más comunes: ", tipo_resumen)

Season                                    int64
Team Name                                object
Conference                               object
Historical Conference                    object
Adjusted Temo                           float64
                                         ...   
Net Rating                              float64
Seed                                     object
Region                                   object
Post-Season Tournament                   object
Post-Season Tournament Sorting Index      int64
Length: 75, dtype: object
Tipos de Datos más comunes:  float64    65
object      8
int64       2
Name: count, dtype: int64


In [6]:
# Vamos a intentar reducir más la cantidad de inputs
NCAA_df = NCAA_df.iloc[:, :-4]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7950, 71)


,Season,Team Name,Conference,Historical Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,...,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,...,6.08,27.92,30.08,18.01,13.00,10.99,John Calipari,14 years,15.0,36.9
1,2021,Gonzaga,West Coast Conference (WCC),West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,...,11.41,23.99,18.56,13.28,24.63,19.49,Mark Few,24 years,25.0,36.5
2,2024,UConn,Big East Conference,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,...,14.05,25.88,17.21,20.98,20.55,15.33,Dan Hurley,5 years,6.0,36.4
3,2008,Kansas,Big 12 Conference,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,...,5.77,26.38,28.48,18.37,13.90,12.87,Bill Self,20 years,21.0,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,...,8.61,23.45,26.61,18.51,17.84,13.59,Tony Bennett,14 years,15.0,34.2


In [7]:
# Seguimos eliminando columnas, ahora vamos con la 3 columna, que es similar a la segunda, Historical Conference
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Historical')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7950, 70)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,16.252900,...,6.08,27.92,30.08,18.01,13.00,10.99,John Calipari,14 years,15.0,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,16.127934,...,11.41,23.99,18.56,13.28,24.63,19.49,Mark Few,24 years,25.0,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,14.499681,...,14.05,25.88,17.21,20.98,20.55,15.33,Dan Hurley,5 years,6.0,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,19.107900,...,5.77,26.38,28.48,18.37,13.90,12.87,Bill Self,20 years,21.0,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,14.706673,...,8.61,23.45,26.61,18.51,17.84,13.59,Tony Bennett,14 years,15.0,34.2


In [8]:
# Seguimos eliminando columnas, ahora vamos con unas columnas que contienen NST que no nos aportan demasiada información
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('NST')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7950, 68)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,16.252900,...,6.08,27.92,30.08,18.01,13.00,10.99,John Calipari,14 years,15.0,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,16.127934,...,11.41,23.99,18.56,13.28,24.63,19.49,Mark Few,24 years,25.0,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,14.499681,...,14.05,25.88,17.21,20.98,20.55,15.33,Dan Hurley,5 years,6.0,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,19.107900,...,5.77,26.38,28.48,18.37,13.90,12.87,Bill Self,20 years,21.0,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,14.706673,...,8.61,23.45,26.61,18.51,17.84,13.59,Tony Bennett,14 years,15.0,34.2


In [9]:
# Seguimos eliminando columnas, ahora vamos con puntos ofensivos y defensivos, ya que nos interesa ir a los porcentajes sobretodo
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('PtFG')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7950, 64)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,16.252900,...,6.08,27.92,30.08,18.01,13.00,10.99,John Calipari,14 years,15.0,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,16.127934,...,11.41,23.99,18.56,13.28,24.63,19.49,Mark Few,24 years,25.0,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,14.499681,...,14.05,25.88,17.21,20.98,20.55,15.33,Dan Hurley,5 years,6.0,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,19.107900,...,5.77,26.38,28.48,18.37,13.90,12.87,Bill Self,20 years,21.0,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,14.706673,...,8.61,23.45,26.61,18.51,17.84,13.59,Tony Bennett,14 years,15.0,34.2


In [10]:
# Seguimos eliminando columnas, ahora vamos con las columnas que contienen Rate
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Rate')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7950, 57)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Current coach,Active Coaching Length,Active Coaching Length Index,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,16.252900,...,6.08,27.92,30.08,18.01,13.00,10.99,John Calipari,14 years,15.0,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,16.127934,...,11.41,23.99,18.56,13.28,24.63,19.49,Mark Few,24 years,25.0,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,14.499681,...,14.05,25.88,17.21,20.98,20.55,15.33,Dan Hurley,5 years,6.0,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,19.107900,...,5.77,26.38,28.48,18.37,13.90,12.87,Bill Self,20 years,21.0,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,14.706673,...,8.61,23.45,26.61,18.51,17.84,13.59,Tony Bennett,14 years,15.0,34.2


In [11]:
# Seguimos eliminando columnas, ahora vamos con unas columnas que contienen información sobre los entrenadores
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Coach|coach')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7950, 54)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,16.252900,...,34.47,20.03,5.69,6.08,27.92,30.08,18.01,13.00,10.99,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,16.127934,...,19.89,6.54,22.61,11.41,23.99,18.56,13.28,24.63,19.49,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,14.499681,...,18.66,20.29,13.83,14.05,25.88,17.21,20.98,20.55,15.33,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,19.107900,...,31.72,22.11,5.59,5.77,26.38,28.48,18.37,13.90,12.87,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,14.706673,...,30.03,14.67,8.82,8.61,23.45,26.61,18.51,17.84,13.59,34.2


In [12]:
# Seguimos eliminando columnas, ahora vamos con las columnas que contienen Pts
NCAA_df = NCAA_df.loc[:, ~NCAA_df.columns.str.contains('Pts|pts')]
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()

Tamaño del dataset: (7950, 49)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,51.511600,16.252900,...,34.47,20.03,5.69,6.08,27.92,30.08,18.01,13.00,10.99,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,61.017812,16.127934,...,19.89,6.54,22.61,11.41,23.99,18.56,13.28,24.63,19.49,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,56.907478,14.499681,...,18.66,20.29,13.83,14.05,25.88,17.21,20.98,20.55,15.33,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,56.591200,19.107900,...,31.72,22.11,5.59,5.77,26.38,28.48,18.37,13.90,12.87,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,55.179961,14.706673,...,30.03,14.67,8.82,8.61,23.45,26.61,18.51,17.84,13.59,34.2


In [13]:
# Vamos a dejar las columnas que sean porcentajes, divididos entre 100
pct_columns = [col for col in NCAA_df.columns if "Pct" in col]

# Paso 2: Dividir los valores de estas columnas por 100
NCAA_df[pct_columns] = NCAA_df[pct_columns] / 100

# El dataframe ahora tiene las columnas "Pct" divididas por 100
NCAA_df.head()

,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Net Rating
0,2015,Kentucky,Southeastern Conference (SEC),62.4,121.3,84.4,17.7,19.6,0.515116,0.162529,...,34.47,20.03,5.69,6.08,27.92,30.08,18.01,13.00,10.99,36.9
1,2021,Gonzaga,West Coast Conference (WCC),73.8,126.4,89.9,14.4,18.0,0.610178,0.161279,...,19.89,6.54,22.61,11.41,23.99,18.56,13.28,24.63,19.49,36.5
2,2024,UConn,Big East Conference,64.6,127.5,91.1,18.6,17.5,0.569075,0.144997,...,18.66,20.29,13.83,14.05,25.88,17.21,20.98,20.55,15.33,36.4
3,2008,Kansas,Big 12 Conference,66.8,120.9,85.7,NaN,NaN,0.565912,0.191079,...,31.72,22.11,5.59,5.77,26.38,28.48,18.37,13.90,12.87,35.2
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,0.551800,0.147067,...,30.03,14.67,8.82,8.61,23.45,26.61,18.51,17.84,13.59,34.2


In [14]:
# Vamos a ver los tipos de nuestras columnas finales
print(NCAA_df.dtypes)
tipo_resumen_v2 = NCAA_df.dtypes.value_counts()
print("Tipos de Datos más comunes: ", tipo_resumen_v2)

Season                               int64
Team Name                           object
Conference                          object
Adjusted Temo                      float64
Adjusted Offensive Efficiency      float64
Adjusted Defensive Efficiency      float64
Avg Possession Length (Offense)    float64
Avg Possession Length (Defense)    float64
eFGPct                             float64
TOPct                              float64
ORPct                              float64
OffFT                              float64
DefFT                              float64
Tempo                              float64
AdjTempo                           float64
OE                                 float64
AdjOE                              float64
DE                                 float64
AdjDE                              float64
AdjEM                              float64
FG2Pct                             float64
FG3Pct                             float64
FTPct                              float64
BlockPct   

In [15]:
# Vamos a ver que variedad de años tenemos, para quedarnos con los últimos años
NCAA_df['Season'].value_counts()

,count
Season,
2023,363
2024,362
2022,358
2021,357
2019,353
2020,353
2015,351
2017,351
2018,351


In [16]:
#Como vemos todos los años están bastante parecidos, y tenemos datos desde el 2002. Vamos a eliminar los datos del 2002, 2003 y 2004 para reducir algo mas el dataset y usar los últimos 20 años
NCAA_df = NCAA_df[~NCAA_df['Season'].isin([2002, 2003, 2004])]
NCAA_df['Season'].value_counts()

,count
Season,
2023,363
2024,362
2022,358
2021,357
2019,353
2020,353
2015,351
2018,351
2014,351


In [17]:
# Vamos a ver el número de líneas que nos quedan
print("Tamaño del dataset:", NCAA_df.shape)

Tamaño del dataset: (6970, 49)


In [18]:
# Vamos a eliminar todas las lineas que tengan algún tipo de caracter en blanco
NCAA_df = NCAA_df.replace(pd.NA, regex=True).dropna()
print("Tamaño del dataset:", NCAA_df.shape)
NCAA_df.head()


Tamaño del dataset: (6966, 49)


,Season,Team Name,Conference,Adjusted Temo,Adjusted Offensive Efficiency,Adjusted Defensive Efficiency,Avg Possession Length (Offense),Avg Possession Length (Defense),eFGPct,TOPct,...,PFOR,SFOR,SGOR,PGOR,CenterDR,PFDR,SFDR,SGDR,PGDR,Net Rating
4,2019,Virginia,Atlantic Coast Conference (ACC),59.4,123.4,89.2,21.0,18.8,0.551800,0.147067,...,30.03,14.67,8.82,8.61,23.45,26.61,18.51,17.84,13.59,34.2
6,2021,Baylor,Big 12 Conference,67.4,125.0,91.1,17.0,17.6,0.565483,0.165283,...,39.18,15.00,7.14,5.21,30.09,27.69,14.23,16.34,11.65,33.9
7,2015,Wisconsin,Big Ten Conference (Big 10),58.7,129.0,95.2,21.8,18.3,0.547518,0.123664,...,28.34,29.46,9.31,11.50,33.03,20.78,22.11,11.68,12.40,33.8
8,2018,Villanova,Big East Conference,68.7,127.8,94.0,17.9,16.6,0.595082,0.150455,...,27.17,19.54,9.99,4.45,29.62,19.16,19.99,17.28,13.94,33.8
9,2011,Ohio State,Big Ten Conference (Big 10),63.8,125.0,91.5,17.1,20.1,0.563137,0.157759,...,18.74,15.64,14.62,9.35,35.97,17.52,15.71,16.02,14.77,33.5


In [19]:
# Por último, vamos a renombrar el df, para tenerlo listo para los siguientes pasos
March_Madness_df = NCAA_df

import os
# Define la ruta a la carpeta
folder_path = "/content/drive/MyDrive/Colab Notebooks/TFM"

# Crea la carpeta si no existe
os.makedirs(folder_path, exist_ok=True)
March_Madness_df.to_csv("/content/drive/MyDrive/Colab Notebooks/TFM/March_Madness_df.csv")
